<a href="https://colab.research.google.com/github/Kotzly/DS4H_Course/blob/dev_e2/notebooks/DS4H_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Kotzly/DS4H_Course.git
!pip install ./DS4H_Course

Cloning into 'DS4H_Course'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 322 (delta 182), reused 164 (delta 71), pack-reused 0
Receiving objects: 100% (322/322), 1.86 MiB | 6.55 MiB/s, done.
Resolving deltas: 100% (182/182), done.
Processing ./DS4H_Course
     |████████████████████████████████| 9.5MB 12.9MB/s 
  Created wheel for ds4h: filename=ds4h-0.1-cp37-none-any.whl size=12046 sha256=99719ab32265c82142da66977ec8867d4b82bbc9795b0c3e35dcb77003f02073
  Stored in directory: /tmp/pip-ephem-wheel-cache-huh6cr8z/wheels/ec/ff/91/fc3664f800ce643b3917437b9c8d3b0b7298db38297e3d6ef4
  Created wheel for zipfile38: filename=zipfile38-0.0.3-cp37-none-any.whl size=22765 sha256=204fc7f6f6af462cd7a02e14a8ef41f702ccf236fc44a1ffd07bb25829e4826a
  Stored in directory: /root/.cache/pip/wheels/15/7a/ec/508d28bf3894e050aef4d0205cdf27a346e6e0891e979345d4
  Created wheel for simpledbf: filename=sim

In [2]:
import ds4h
from ds4h.data.download_sinasc import download_dbc, download_dbf_2020
from ds4h.processing.dbc import dbc_to_csv, dbf_to_csv
from ds4h.processing.csv import join_sinasc_files
from ds4h.processing.sinasc import process_sinasc

from pathlib import Path
from ds4h.data.city_ranking import get_ranking_dataframe, get_top_cities_from_df

In [3]:
dataset_root = Path("/home/paulo/Documents/datasets/sinasc")

In [4]:
dbc_path = dataset_root / "dbc"

dbc_path.mkdir(exist_ok=True, parents=True)

In [6]:
download_dbc(dbc_path, n_jobs=10, years=[2019])

URLError: ignored

In [ ]:
csv_path = dataset_root / "csv"
dbc_to_csv(dbc_path, csv_path);

In [ ]:
dbf_folder = dataset_root / "dbf"
dbf_folder.mkdir(exist_ok=True)

download_dbf_2020(dbf_folder)

dbf_filepath = list(dbf_folder.glob("*.DBF"))[0]
dbf_to_csv(dbf_filepath, csv_path)

In [ ]:
union_df = join_sinasc_files(csv_path)
union_df

In [ ]:
df = get_ranking_dataframe()
cities_dict = get_top_cities_from_df(df)
print(cities_dict)

In [ ]:
df = process_sinasc(union_df)

In [ ]:
import numpy as np
def null_proportion(df):
  def null_prop(series):
    mp = np.isnan(series.values).mean()
    return "{:.1f}%".format(mp * 100)
  return df.apply(null_prop, axis=0)

null_proportion(union_df)

In [ ]:
union_df.head()

In [ ]:
raw_df = union_df

In [ ]:
df["YEAR"] = df.DTNASC.dt.year
df["MONTH"] = df.DTNASC.dt.month
df["DAY"] = df.DTNASC.dt.day
df

In [ ]:
a,b = "YEAR", "MUNNAME"
pc = (100 * df.groupby([a, b]).size() / df.groupby([a]).size()).reset_index().rename({0:'percent'}, axis=1)
# df.groupby(["MUNNAME", "ESCMAE"]).size() / df.groupby(["MUNNAME"]).size()
sns.barplot(data=pc, x=a, hue=b, y="percent")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def pct_plot(df, x_col, group_col, percentage=True, y_logscale=False):
  plt.figure(figsize=(14, 5))
  ax = plt.gca()
  ax.grid()
  if percentage:
    pct2 = (100 * df.groupby([x_col, group_col]).size() / df.groupby([x_col]).size()).reset_index().rename({0:'percent'}, axis=1)    
    plot = sns.barplot(x=x_col, hue=group_col, y='percent', data=pct2, ax=ax)
    ticks = ax.get_yticks()
    if y_logscale:
      ax.set_yscale("log")
    ax.set_yticklabels([f"{x}%" for x in ticks])
  else:
    plot = sns.countplot(data=df, x=x_col, hue=group_col, ax=ax)
    if y_logscale:
      ax.set_yscale("log")

  ax.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
  ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)


  plt.show()
  plt.close()

In [ ]:
sns.countplot(data=df, x="YEAR")

In [ ]:
pct_plot(df, "MUNNAME", "ESCMAE", percentage=True)

In [ ]:
pct_plot(df, "MUNNAME", "RACACOR", percentage=True)

In [ ]:
pct_plot(df, "YEAR", "MUNNAME", percentage=True, y_logscale=True)

In [ ]:
pct_plot(df, "YEAR", "RACACOR", percentage=True)

In [ ]:
pct_plot(df, "YEAR", "ESTCIVMAE", percentage=True)

In [ ]:
pct_plot(df, "YEAR", "ESCMAE", percentage=True)

In [ ]:
pct_plot(df, "YEAR", "PARTO", percentage=True)

In [ ]:
pct_plot(df, "YEAR", "ESTCIVMAE", percentage=True)

In [ ]:
pct_plot(df, "MONTH", "YEAR", percentage=False)

# 2020

In [ ]:
df2020 = df[df.YEAR == 2020]

In [ ]:
pct_plot(df2020, "MONTH", "PARTO", percentage=True)

In [ ]:
pct_plot(df2020, "MONTH", "RACACOR", percentage=False)

In [ ]:
pct_plot(df2020, "MONTH", "ESCMAE", percentage=False)

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi
from statsmodels.formula.api import ols

In [ ]:
pearson_pairs = [
  ["IDADEMAE"]
]
spearman_pairs = [
  ["IDADEMAE", "ESCMAE"],
  ["IDADEMAE", "QTDFILVIVO"],
  ["IDADEMAE", "QTDFILMORTO"],
  ["IDADEMAE", "GESTACAO"],
]
cat_num_pairs = [
  "DTNASC"
]
cat_pairs = [
  ["PARTO", "YEAR"],
  ["PARTO", "YEAR"],
  ["PARTO", "YEAR"],
  
]
raw_df.corr().loc[
  ["DTNASC", "IDADEMAE"]
]

In [ ]:
union_df.groupby(["DTNASC.dt.year"]).count()["DTNASC"]
